#### 0.思路说明如下：
调用MLR包（一个R中非常全面的机器学习包，包含回归、分类、生存分析、多标签等模型，可以调用一般算法，可以封装MLR包暂时尚未直接调用的算法，甚至可以直接调用h2o深度学习框架，使用说明文档：http://mlr-org.github.io/mlr-tutorial/devel/html/ ）封装lightgbm算法，自动调参。具体使用步骤如下（此处只涉及多分类模型，回归、二分类、多标签分类可以模仿封装）：

#### 1.加装相关软件包

In [1]:
library(lightgbm)
library(mlr)
library(dplyr)

Loading required package: R6
Loading required package: ParamHelpers

Attaching package: 'mlr'

The following object is masked from 'package:MicrosoftML':

    selectFeatures


Attaching package: 'dplyr'

The following object is masked from 'package:lightgbm':

    slice

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



#### 2.封装函数
说明如下：
1）具体使用方法请参考MLR包官方网址 http://mlr-org.github.io/mlr-tutorial/devel/html/ 中的Appendix部分，下面脚本中尽可能的添加了多分类使用说明，请查看后使用，以防错误；此处脚本未实现完全自动调用；  
2）训练模型时设置验证数据集。

In [2]:
makeRLearner.classif.lgb = function() {
    makeRLearnerClassif(
        cl = "classif.lgb",
        package = "lightgbm",
        par.set = makeParamSet(
            makeIntegerLearnerParam(id = "num_iterations", default = 100, lower=1), 
            makeIntegerLearnerParam(id = "nrounds", default = 10, lower = 1, upper = 1e4), 
            makeIntegerLearnerParam(id = "verbose", default = 1),
            makeDiscreteLearnerParam(id = "boosting", default = "gbdt", values = c("gbdt", "rf", "dart", "goss")), 
            makeNumericLearnerParam(id = "learning_rate", default = 0.1, lower = 0),
            makeIntegerLearnerParam(id = "max_depth", default = -1, lower = -1),  
            makeIntegerLearnerParam(id = "min_data_in_leaf", default = 20, lower = 0), 
            makeIntegerLearnerParam(id = "num_leaves", default = 31, lower = 1),
            makeNumericLearnerParam(id = "feature_fraction", default = 1, lower = 0, upper = 1), 
            makeNumericLearnerParam(id = "bagging_fraction", default = 1, lower = 0, upper = 1),
            makeIntegerLearnerParam(id = "bagging_freq", default = 0, lower = 0), 
            makeNumericLearnerParam(id = "min_gain_to_split", default = 0, lower = 0),
            makeNumericLearnerParam(id = "min_sum_hessian", default = 10),
            makeIntegerLearnerParam(id = "early_stopping_round", default = 0, lower = 0),
            makeNumericLearnerParam(id = "lambda_l1", default = 0, lower = 0),
            makeNumericLearnerParam(id = "lambda_l2", default = 0, lower = 0), 
            makeNumericLearnerParam(id = "drop_rate", default = 0.1, lower = 0, upper = 1, requires = quote(boosting == "dart")),
            makeIntegerLearnerParam(id = "max_drop", default = 50, requires = quote(boosting == "dart")),
            makeLogicalLearnerParam(id = "uniform_drop", default = FALSE, requires = quote(boosting == "dart")),
            makeLogicalLearnerParam(id = "xgboost_dart_mode", default = FALSE, requires = quote(boosting == "dart")),
            makeNumericLearnerParam(id = "top_rate", default = 0.2, lower = 0.1, upper = 1, requires = quote(boosting == "goss")),
            makeNumericLearnerParam(id = "other_rate", default = 0.1, lower = 0, upper = 1, requires = quote(boosting == "goss")),
            makeIntegerLearnerParam(id = "max_cat_group", default = 64),
            makeIntegerLearnerParam(id = "max_cat_threshold", default = 256),
            makeNumericLearnerParam(id = "min_cat_smooth", default = 5),
            makeNumericLearnerParam(id = "max_cat_smooth", default = 100),
            makeNumericLearnerParam(id = "cat_smooth_ratio", default = 0.01),
            ## 用于分类变量，可以减小分类变量中噪音的影响，特别是类别中数据较少的分别变量
            makeIntegerLearnerParam(id = "max_bin", default = 255),
            ## 数据行分组数，少的分组数会降低训练精度（训练速度快），但是可以提升泛化能力
            makeIntegerLearnerParam(id = "min_data_in_bin", default = 5),
            ## 分组内最小用户数
            makeLogicalLearnerParam(id = "header", default = FALSE),
            ## 设置为TRUE表示数据有字段名
            makeLogicalLearnerParam(id="use_missing", default = TRUE,tunable = FALSE),
            ## 设置为FALSE将不能对缺失值进行特殊操作 
            makeLogicalLearnerParam(id = "zero_as_missing", default = FALSE),
            ## 是否用0代替缺失值与未知值
            makeIntegerLearnerParam(id = "num_class", default = 5, lower = 1, upper = 1e6),
            ## 声明多分类中的类数
            makeDiscreteLearnerParam(id = "metric", default = "multi_logloss", values = c("multi_logloss", "multi_error")),
            makeLogicalLearnerParam(id = "is_training_metric", default = TRUE),
            ## set this to TRUE if need to output metric result of training
            makeDiscreteLearnerParam(id = "objective", default = "multiclass", values = c("multiclass", "binary")),
            makeDiscreteLearnerParam(id = "predict.method", values = c("plug-in", "predictive", "debiased"), default = "plug-in", when = "predict")
        ),
        par.vals = list(objective = "multiclass"),## 此处请按个人需求修改
        properties = c("multiclass", "numerics", "factors", "prob", "weights"), 
        name = "LightGBM",
        short.name = "lgb",
        note = "Learner param 'predict.method' maps to 'method' in predict.lgb."
    )
}

trainLearner.classif.lgb = function(.learner, .task, .subset, .weights = NULL,  ...) {
    data = getTaskData(.task, .subset, target.extra = TRUE)
    lgb.data1 = lgb.Dataset(
        data = as.matrix(data$data[1:dim(data_train)[1], ]), ## 请将此处data_train修改为实际训练数据集的名称,见第3步
        label = as.integer(data$target[1:dim(data_train)[1]]) - 1L ## 请将此处data_train修改为实际训练数据集的名称
    )
    lgb.data2 = lgb.Dataset.create.valid(
        dataset = lgb.data1,
        data = as.matrix(data$data[(dim(data_train)[1] + 1):dim(data_total)[1], ]), 
        ## 请将此处data_train修改为实际训练数据集的名称，data_total是指训练数据+验证数据集的名称，即所有数据,见第3步
        label = as.integer(data$target[(dim(data_train)[1] + 1):dim(data_total)[1]]) - 1L
        ## 请将此处data_train修改为实际训练数据集的名称，data_total是指训练数据+验证数据集的名称，即所有数据,见第3步
    )
    valids = list(test = lgb.data2)
    
    lightgbm::lgb.train(
        list(), 
        data = lgb.data1,
        valids = valids,
        num_threads = 4, ## 请按实际需要的电脑线程数进行修改，默认值为CPU核数，非线程数（譬如本人电脑为两核四线程，则默认值为2）
        verbose = 1,
        record = TRUE,
        #early_stopping_rounds = 10,
        nrounds = 1, ## cross-validation,不进行交叉验证
        ...
    )
}

predictLearner.classif.lgb = function(.learner, .model, .newdata, ...) {
    p = predict(.model$learner.model, data.matrix(.newdata), reshape = TRUE, ...)
    colnames(p) <- 0:4
    return(p)
}

registerS3method("makeRLearner", "classif.lgb", makeRLearner.classif.lgb)
registerS3method("trainLearner", "classif.lgb", trainLearner.classif.lgb)
registerS3method("predictLearner", "classif.lgb", predictLearner.classif.lgb)


#### 3.读取数据

In [3]:
train <- read.table(file = 'C:/Users/Administrator/LightGBM/examples/multiclass_classification/multiclass.train') 
set.seed(1)
train_s <- sample(dim(train)[1], round(dim(train)[1] * .9, 0)) ## lightgbm自带训练数据随机分割为两部分（如果实际建模因变量类别不平衡，请修改为分层抽样）
data_train <- train[train_s, ] ## 训练数据集
data_validation <- train[-train_s, ]## 验证数据集
rm(train) ## 删除，以减少内存
gc()
data_test <- read.table(file = 'C:/Users/Administrator/LightGBM/examples/multiclass_classification/multiclass.test') ## 测试数据集
data_total <- rbind(data_train, data_validation) ## 上面封装函数中的“data_total”
data_total$V1 <- as.factor(data_total$V1) ## 对于分类模型，MLR包要求因变量的数据类型为factor

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1590652,85.0,2637877,140.9,2164898,115.7
Vcells,2305799,17.6,3851194,29.4,3142118,24.0


#### 4.构建任务
MLR包可以把训练数据集与验证数据集一起放在任务中

In [4]:
lgb.task <- makeClassifTask(data = data_total, target = "V1") 

#### 5.设置需要调整的参数
此处为示例，实际调参中需要调整lightgbm算法的很多参数

In [5]:
lgb.ps <- makeParamSet(
    makeNumericParam("learning_rate", lower = .1, upper = 1),
    makeDiscreteParam("num_iterations", values = 100), ##默认值为100
    makeIntegerParam("num_leaves", lower = 5, upper = 50)
)

#### 6.学习次数控制
参数resolution = 10L意思是learning_rate为.1, .2, .3, ……, 1, 共10个

In [6]:
lgb.ctrl <- makeTuneControlGrid(resolution = 10L) ## 默认值为10

#### 7.模型描述--重复抽样设置
自动调参必须设置此参数（郁闷之处），其它重抽样方法可以查看函数makeResampleDesc的使用说明

In [7]:
lgb.rdesc <- makeResampleDesc(method = "Bootstrap", iters = 10, stratify = TRUE) 
## 重抽样10次

#### 8.构建学习器

In [8]:
lrn.ps <- list(
    objective = "multiclass",
    metric = "multi_logloss",
    num_class = 5, ##请根据实际情况进行调整
    header = TRUE
)
lgb.lrn = makeLearner(
    'classif.lgb', 
    predict.type = 'prob', ## 输出的为概率，不是分类结果，用来计算logloss
    par.vals = lrn.ps
)

##### 9.调参

In [9]:
res_time <- system.time(
    lgb.res <- tuneParams(
        learner = lgb.lrn,
        task = lgb.task,
        resampling = lgb.rdesc, 
        par.set = lgb.ps,
        control = lgb.ctrl,
        show.info = TRUE,
        measures = list(logloss)
    )
)
res_time ## 175秒

[Tune] Started tuning learner classif.lgb for parameter set:
                   Type len Def   Constr Req Tunable Trafo
learning_rate   numeric   -   - 0.1 to 1   -    TRUE     -
num_iterations discrete   -   -      100   -    TRUE     -
num_leaves      integer   -   -  5 to 50   -    TRUE     -
With control class: TuneControlGrid
Imputation value: Inf
[Tune-x] 1: learning_rate=0.1; num_iterations=100; num_leaves=5


[1]:	test's multi_logloss:1.63238 
[1]:	test's multi_logloss:1.62819 
[1]:	test's multi_logloss:1.6301 
[1]:	test's multi_logloss:1.62685 
[1]:	test's multi_logloss:1.63074 
[1]:	test's multi_logloss:1.63175 
[1]:	test's multi_logloss:1.63043 
[1]:	test's multi_logloss:1.63004 
[1]:	test's multi_logloss:1.63099 
[1]:	test's multi_logloss:1.63019 


[Tune-y] 1: logloss.test.mean=1.61; time: 0.0 min; memory: 116Mb use, 172Mb max
[Tune-x] 2: learning_rate=0.2; num_iterations=100; num_leaves=5


[1]:	test's multi_logloss:1.65609 
[1]:	test's multi_logloss:1.64774 
[1]:	test's multi_logloss:1.6515 
[1]:	test's multi_logloss:1.64502 
[1]:	test's multi_logloss:1.65278 
[1]:	test's multi_logloss:1.65495 
[1]:	test's multi_logloss:1.65209 
[1]:	test's multi_logloss:1.6514 
[1]:	test's multi_logloss:1.65336 
[1]:	test's multi_logloss:1.65171 


[Tune-y] 2: logloss.test.mean= 1.6; time: 0.0 min; memory: 114Mb use, 172Mb max
[Tune-x] 3: learning_rate=0.3; num_iterations=100; num_leaves=5


...

[1]:	test's multi_logloss:1.6722 
[1]:	test's multi_logloss:1.59016 
[1]:	test's multi_logloss:1.68676 
[1]:	test's multi_logloss:1.61223 
[1]:	test's multi_logloss:1.71799 
[1]:	test's multi_logloss:1.60586 
[1]:	test's multi_logloss:1.64489 
[1]:	test's multi_logloss:1.67651 
[1]:	test's multi_logloss:1.70119 
[1]:	test's multi_logloss:1.64547 


[Tune-y] 99: logloss.test.mean=1.63; time: 0.0 min; memory: 116Mb use, 181Mb max
[Tune-x] 100: learning_rate=1; num_iterations=100; num_leaves=50


[1]:	test's multi_logloss:1.69631 
[1]:	test's multi_logloss:1.60781 
[1]:	test's multi_logloss:1.71358 
[1]:	test's multi_logloss:1.63233 
[1]:	test's multi_logloss:1.74922 
[1]:	test's multi_logloss:1.62342 
[1]:	test's multi_logloss:1.66589 
[1]:	test's multi_logloss:1.70321 
[1]:	test's multi_logloss:1.73018 
[1]:	test's multi_logloss:1.66887 


[Tune-y] 100: logloss.test.mean=1.65; time: 0.0 min; memory: 116Mb use, 181Mb max
[Tune] Result: learning_rate=0.4; num_iterations=100; num_leaves=45 : logloss.test.mean=1.58


   user  system elapsed 
 337.31  134.90  178.31 

#### 10.查看最优结果

In [10]:
lgb.res

Tune result:
Op. pars: learning_rate=0.4; num_iterations=100; num_leaves=45
logloss.test.mean=1.58

#### 11.查看训练过中所有参数组合对应的交叉熵

In [11]:
lgb.opt.path <- as.data.frame(trafoOptPath(lgb.res$opt.path))
lgb.opt.path  %>% arrange(logloss.test.mean) %>% head(5)

learning_rate,num_iterations,num_leaves,logloss.test.mean,dob,eol,error.message,exec.time
0.4,100,45,1.584683,84,NA,NA,1.47
0.4,100,40,1.584914,74,NA,NA,1.34
0.4,100,50,1.584957,94,NA,NA,1.48
0.4,100,35,1.584984,64,NA,NA,1.39
0.5,100,35,1.585737,65,NA,NA,1.39


#### 12.预测
即用最优参数训练模型，对测试数据进行预测，具体如下：  
1）构建训练模型用的函数（用mlr封装lightgbm算法）；  
2）构建学习任务；  
3）设定参数；  
4）设定学习器；  
5）训练模型；  
6）预测；  
7）计算multilogloss；  
具体脚本如下：

In [12]:
makeRLearner.classif.lgb2 = function() {
    makeRLearnerClassif(
        cl = "classif.lgb",
        package = "lightgbm",
        par.set = makeParamSet(
            makeIntegerLearnerParam(id = "num_iterations", default = 100, lower=1), 
            makeIntegerLearnerParam(id = "nrounds", default = 10, lower = 1, upper = 1e4), 
            makeIntegerLearnerParam(id = "verbose", default = 1),
            makeDiscreteLearnerParam(id = "boosting", default = "gbdt", values = c("gbdt", "rf", "dart", "goss")), 
            makeNumericLearnerParam(id = "learning_rate", default = 0.1, lower = 0),
            makeIntegerLearnerParam(id = "max_depth", default = -1, lower = -1),  
            makeIntegerLearnerParam(id = "min_data_in_leaf", default = 20, lower = 0), 
            makeIntegerLearnerParam(id = "num_leaves", default = 31, lower = 1),
            makeNumericLearnerParam(id = "feature_fraction", default = 1, lower = 0, upper = 1), 
            makeNumericLearnerParam(id = "bagging_fraction", default = 1, lower = 0, upper = 1),
            makeIntegerLearnerParam(id = "bagging_freq", default = 0, lower = 0), 
            makeNumericLearnerParam(id = "min_gain_to_split", default = 0, lower = 0),
            makeNumericLearnerParam(id = "min_sum_hessian", default = 10),
            makeIntegerLearnerParam(id = "early_stopping_round", default = 0, lower = 0),
            makeNumericLearnerParam(id = "lambda_l1", default = 0, lower = 0),
            makeNumericLearnerParam(id = "lambda_l2", default = 0, lower = 0), 
            makeNumericLearnerParam(id = "drop_rate", default = 0.1, lower = 0, upper = 1, requires = quote(boosting == "dart")),
            makeIntegerLearnerParam(id = "max_drop", default = 50, requires = quote(boosting == "dart")),
            makeLogicalLearnerParam(id = "uniform_drop", default = FALSE, requires = quote(boosting == "dart")),
            makeLogicalLearnerParam(id = "xgboost_dart_mode", default = FALSE, requires = quote(boosting == "dart")),
            makeNumericLearnerParam(id = "top_rate", default = 0.2, lower = 0.1, upper = 1, requires = quote(boosting == "goss")),
            makeNumericLearnerParam(id = "other_rate", default = 0.1, lower = 0, upper = 1, requires = quote(boosting == "goss")),
            makeIntegerLearnerParam(id = "max_cat_group", default = 64),
            makeIntegerLearnerParam(id = "max_cat_threshold", default = 256),
            makeNumericLearnerParam(id = "min_cat_smooth", default = 5),
            makeNumericLearnerParam(id = "max_cat_smooth", default = 100),
            makeNumericLearnerParam(id = "cat_smooth_ratio", default = 0.01),
            ## 用于分类变量，可以减小分类变量中噪音的影响，特别是类别中数据较少的分别变量
            makeIntegerLearnerParam(id = "max_bin", default = 255),
            ## 数据行分组数，少的分组数会降低训练精度（训练速度快），但是可以提升泛化能力
            makeIntegerLearnerParam(id = "min_data_in_bin", default = 5),
            ## 分组内最小用户数
            makeLogicalLearnerParam(id = "header", default = FALSE),
            ## 设置为TRUE表示数据有字段名
            makeLogicalLearnerParam(id="use_missing", default = TRUE,tunable = FALSE),
            ## 设置为FALSE将不能对缺失值进行特殊操作 
            makeLogicalLearnerParam(id = "zero_as_missing", default = FALSE),
            ## 是否用0代替缺失值与未知值
            makeIntegerLearnerParam(id = "num_class", default = 9, lower = 1, upper = 1e6),
            ## 声明多分类中的类数
            makeDiscreteLearnerParam(id = "metric", default = "multi_logloss", values = c("multi_logloss", "multi_error")),
            makeLogicalLearnerParam(id = "is_training_metric", default = TRUE),
            ## set this to TRUE if need to output metric result of training
            makeDiscreteLearnerParam(id = "objective", default = "multiclass", values = c("multiclass", "binary")),
            makeDiscreteLearnerParam(id = "predict.method", values = c("plug-in", "predictive", "debiased"), default = "plug-in", when = "predict")
        ),
        par.vals = list(objective = "multiclass"),## 此处请按个人需求修改
        properties = c("multiclass", "numerics", "factors", "prob", "weights"), 
        name = "LightGBM",
        short.name = "lgb",
        note = "Learner param 'predict.method' maps to 'method' in predict.lgb."
    )
}

trainLearner.classif.lgb = function(.learner, .task, .subset, .weights = NULL,  ...) {
    data = getTaskData(.task, .subset, target.extra = TRUE)
    lgb.data1 = lgb.Dataset(
        data = as.matrix(data$data), ## 请将此处data_train修改为实际训练数据集的名称,见第3步
        label = as.integer(data$target) - 1L ## 请将此处data_train修改为实际训练数据集的名称
    )
    
    lightgbm::lgb.train(
        list(), 
        data = lgb.data1,
        num_threads = 4, ## 请按实际需要的电脑线程数进行修改，默认值为CPU核数，非线程数（譬如本人电脑为两核四线程，则默认值为2）
        verbose = 1,
        record = TRUE,
        #objective = "multiclass",
        #metric = "multi_logloss",
        #num_class = 5,
        ...
    )
}

predictLearner.classif.lgb = function(.learner, .model, .newdata, ...) {
    p = predict(.model$learner.model, data.matrix(.newdata), reshape = TRUE, ...)
    colnames(p) <- 0:4
    return(p)
}

registerS3method("makeRLearner", "classif.lgb", makeRLearner.classif.lgb)
registerS3method("trainLearner", "classif.lgb", trainLearner.classif.lgb)
registerS3method("predictLearner", "classif.lgb", predictLearner.classif.lgb)


In [13]:
data_total2 <- rbind(data_train, data_test)
## 构建任务
lgb.task <- makeClassifTask(id = "multi", data = data_total2, target = "V1")

## 参数
lgb.ps <- list(
    objective = "multiclass",
    metric = "multi_logloss",
    num_class = 5,
    header = TRUE,
    learning_rate = .4, ## 修改为tuneParams训练出来的最优参数
    num_iterations = 100, 
    num_leaves = 45  ## 修改为tuneParams训练出来的最优参数
)

## 学习器
lgb.lrn <- makeLearner(
    "classif.lgb2", 
    predict.type = "prob",
    par.vals = lgb.ps
)

## 训练
lgb.train1 <- train(
    learner = lgb.lrn,
    task = lgb.task,
    subset = 1:dim(data_train)[1]
)

In [14]:
## 预测测试集
t_data <- getTaskData(lgb.task, subset = (dim(data_train)[1] + 1):dim(data_total2)[1], target.extra = TRUE)$data
lgb.pred_mlr <- predict(lgb.train1, newdata = t_data)
lgb.pred_mlr_data = as.matrix(lgb.pred_mlr$data[, 1:5])

## 计算multilogloss
library(MLmetrics)
MultiLogLoss(lgb.pred_mlr_data, data_test$V1)


Attaching package: 'MLmetrics'

The following object is masked from 'package:base':

    Recall



[1] 1.379644

#### 13.注：
1）本人测试以上程序所使用电脑为win10，两核四线程；  
2）本人所使用的lightgbm为CPU版本，未调用GPU；  
3) 本人所测试程序为R版本lightgbm，本人所用R为Microsoft R Server9.1.0版本；  
4）本人运行循环程序时长为334秒；  
5）栅格设置处，learning_rate/num_leaves的每组值之间的间距设置不合理，例如learning的每个值之间的间距随着learning_rate的增加而增加更为合理，此处设置只是为了说明问题而已，请根据实际情况决定，勿采纳！  
6）第一个lightgbm的封装函数可以设置nrounds=10进行10折交叉验证，但是使用tuneParams函数时必须设置重抽样方法；因此封装lightgbm时设置nrounds=1，在makeResampleDesc函数中设置iters=10。